In [1]:
import gymnasium as gym
import os

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv

from envs.curriculum_halfcheetah import HalfCheetahEnv

from policy.curriculum_ppo import CurriculumPPO

2023-10-31 18:36:15.802097: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 18:36:15.802128: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 18:36:15.802144: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 18:36:15.806349: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def make_env(env_id: str, rank: int, task = None, seed: int = 0):
    """
    Utility function for multiprocessed env.

    :param env_id: the environment ID
    :param num_env: the number of environments you wish to have in subprocesses
    :param seed: the inital seed for RNG
    :param rank: index of the subprocess
    """
    def _init():
        env = gym.make(env_id)
        check_env(env) # check the environment
        env.reset(seed=seed + rank)
        return env
    set_random_seed(seed)
    return _init

In [3]:
env_id = "Curriculum/HalfCheetah-v4"
num_cpu = 4

In [4]:
from gymnasium.envs.registration import register

register(
     id="Curriculum/HalfCheetah-v4",
     entry_point="envs.curriculum_halfcheetah:HalfCheetahEnv",
     max_episode_steps=1000,
)

/home/kh-ryu/anaconda3/envs/curriculum/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment Curriculum/HalfCheetah-v4 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [5]:
training_env = SubprocVecEnv([make_env(env_id, i, task=None) for i in range(num_cpu)])

2023-10-31 18:36:18.747538: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 18:36:18.747574: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 18:36:18.747589: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 18:36:18.747714: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 18:36:18.747733: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register f

ConnectionResetError: [Errno 104] Connection reset by peer

In [ ]:
obs = training_env.reset()

In [ ]:
model = CurriculumPPO("MlpPolicy", training_env, task_list=None, task_threshold=None, verbose=1, tensorboard_log="./tensorboard/")

In [ ]:
for _ in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = training_env.step(action)
    print(dones)
    print(_)
    if dones.any():
        break